#### Install Web Scraper

In [13]:
! easy_install beautifulsoup4

Searching for beautifulsoup4
Best match: beautifulsoup4 4.7.1
Adding beautifulsoup4 4.7.1 to easy-install.pth file

Using c:\users\edson\anaconda3\lib\site-packages
Processing dependencies for beautifulsoup4
Finished processing dependencies for beautifulsoup4


#### Install Dependencies for HTML5

In [14]:
! easy_install html5lib

Searching for html5lib
Best match: html5lib 1.0.1
Adding html5lib 1.0.1 to easy-install.pth file

Using c:\users\edson\anaconda3\lib\site-packages
Processing dependencies for html5lib
Finished processing dependencies for html5lib


In [16]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


#### Import Dependences

In [17]:
import pandas as pd
import requests
import numpy as np
import io
import requests
from bs4 import BeautifulSoup
import geocoder # import geocoder

In [18]:
canadaRequest = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
canadaSoup = BeautifulSoup(canadaRequest.text, "html5lib")
#create var for df
columnNames = []
#read Headers of table
for head in canadaSoup.table.tbody.tr:
    if (head.string!='\n'):
        columnNames.append(head.string.replace('\n',''))
dfCanada = pd.DataFrame(columns=columnNames)
dfCanada

,Postcode,Borough,Neighbourhood


#### Fill data from Table

In [19]:
# Fill Data from Wikipedia Page
table_body = canadaSoup.find('tbody')
rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    if (len(cols)!=0):
        dfCanada = dfCanada.append({
            'Postcode' : cols[0],
            'Borough': cols[1],
            'Neighbourhood' : cols[2]
        }, ignore_index=True)

#### Clean Data

In [20]:
# filter data frame without "Not Assigned" -> Borough
dfCanadaClean = dfCanada['Borough']!='Not assigned'
dfCanada = dfCanada[dfCanadaClean].reset_index(drop=True)
dfCanada.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [21]:
# Group by and append
dfJoinedData = dfCanada.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: ','.join(x)).reset_index()
dfJoinedData.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [84]:
# Clean 'Neigbourhood' with -> Not assigned data
dfJoinedData.loc[dfJoinedData.Neighbourhood == 'Not assigned','Neighbourhood'] = dfJoinedData.Borough
# Show the fixed Value
dfJoinedData[dfJoinedData.Borough == "Queen's Park"]

,Postcode,Borough,Neighbourhood,Latitude,Longitude
85,M7A,Queen's Park,Queen's Park,NaN,NaN


In [51]:
#Import Data from CSV (Because no connection with geocoder or takes too much)
url="https://cocl.us/Geospatial_data"
csvData=requests.get(url).content
torontoGeoData=pd.read_csv(io.StringIO(csvData.decode('utf-8')))
torontoGeoData = torontoGeoData.rename(columns={'Postal Code': 'Postalcode'})
torontoGeoData.head()

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [ ]:
for index in range (0 , len(dfJoinedData)-2):
    postCode = dfJoinedData.loc[index,'Postcode']
    valueObtained = torontoGeoData.loc[torontoGeoData['Postalcode'] == postCode]
    if (value is not None):
        dfJoinedData.loc[index]['Postcode'] = valueObtained.Latitude

C:\Users\Edson\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [89]:
dfJoinedData

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",NaN,NaN
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",NaN,NaN
3,M1G,Scarborough,Woburn,NaN,NaN
4,M1H,Scarborough,Cedarbrae,NaN,NaN
5,M1J,Scarborough,Scarborough Village,NaN,NaN
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",NaN,NaN
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",NaN,NaN
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",NaN,NaN
9,M1N,Scarborough,"Birch Cliff,Cliffside West",NaN,NaN
